<a href="https://colab.research.google.com/github/Saby-Bishops/Hands-on_Feb11/blob/main/Streamlit_RAG_CodeExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlit-based RAG with Wikipedia Data

In [1]:
!pip install streamlit wikipedia-api sentence-transformers pyngrok


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━

In [2]:
from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saniyapandita","key":"715ac33b0fc80a49c939d524474017e3"}'}

In [3]:
!pip install kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!pip install kaggle

In [15]:
from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"saniyapandita","key":"715ac33b0fc80a49c939d524474017e3"}'}

In [16]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [17]:
!kaggle datasets download -d nikhileswarkomati/suicide-watch

Dataset URL: https://www.kaggle.com/datasets/nikhileswarkomati/suicide-watch
License(s): CC-BY-SA-4.0
 99% 60.0M/60.6M [00:05<00:00, 14.8MB/s]
100% 60.6M/60.6M [00:05<00:00, 12.3MB/s]


In [18]:
!unzip suicide-watch.zip

Archive:  suicide-watch.zip
  inflating: Suicide_Detection.csv   


In [20]:
!ls

 amazonreviews.zip   kaggle.json   Suicide_Detection.csv   test.ft.txt	     train.ft.txt
'kaggle (1).json'    sample_data   suicide-watch.zip	   test.ft.txt.bz2   train.ft.txt.bz2


In [21]:
import pandas as pd

# Load the dataset
df = pd.read_csv('Suicide_Detection.csv')

# Display the first few rows
print(df.head())

   Unnamed: 0                                               text        class
0           2  Ex Wife Threatening SuicideRecently I left my ...      suicide
1           3  Am I weird I don't get affected by compliments...  non-suicide
2           4  Finally 2020 is almost over... So I can never ...  non-suicide
3           8          i need helpjust help me im crying so hard      suicide
4           9  I’m so lostHello, my name is Adam (16) and I’v...      suicide


In [22]:
print(df.columns)

Index(['Unnamed: 0', 'text', 'class'], dtype='object')


In [23]:
import re

# Clean the text
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))  # Remove special characters
df['text'] = df['text'].str.lower()  # Convert to lowercase

# Handle missing values (if any)
df['text'] = df['text'].fillna('')  # Replace NaN with empty strings

# Display the cleaned data
print(df.head())

   Unnamed: 0                                               text        class
0           2  ex wife threatening suiciderecently i left my ...      suicide
1           3  am i weird i dont get affected by compliments ...  non-suicide
2           4  finally 2020 is almost over so i can never hea...  non-suicide
3           8          i need helpjust help me im crying so hard      suicide
4           9  im so losthello my name is adam 16 and ive bee...      suicide


In [24]:
!pip install sentence-transformers faiss-cpu streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 52.4 MB/s eta 0:00:00


In [25]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(df['text'].tolist())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [26]:
import faiss
import numpy as np

# Create a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

NameError: name 'embeddings' is not defined

In [ ]:
def retrieve_info(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return df.iloc[indices[0]]

In [ ]:
results = retrieve_info("I feel so alone")
print(results)

In [27]:
import streamlit as st
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# Load the dataset
@st.cache_data  # Cache the dataset to avoid reloading on every run
def load_dataset():
    df = pd.read_csv('Suicide_Detection.csv')
    return df

# Load the Sentence Transformer model and generate embeddings
@st.cache_resource  # Cache the model to avoid reloading on every run
def load_model_and_embeddings(df):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(df['text'].tolist())
    return model, embeddings

# Build the FAISS index
def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index

# Retrieve relevant information
def retrieve_info(query, model, index, df, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return df.iloc[indices[0]]

# Streamlit app
def main():
    st.title("Suicide and Depression Detection RAG System")

    # Load the dataset
    df = load_dataset()

    # Load the model and generate embeddings
    model, embeddings = load_model_and_embeddings(df)

    # Build the FAISS index
    index = build_faiss_index(embeddings)

    # User input
    query = st.text_input("Enter your query:")
    if st.button("Search"):
        if query:
            # Retrieve relevant information
            results = retrieve_info(query, model, index, df)
            st.write("Search Results:")
            st.dataframe(results)
        else:
            st.warning("Please enter a query.")

# Run the app
if __name__ == "__main__":
    main()

2025-02-12 00:04:01.771 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-02-12 00:04:01.776 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-12 00:04:01.966 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-12 00:04:01.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-12 00:04:01.973 No runtime found, using MemoryCacheStorageManager
2025-02-12 00:04:01.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-12 00:04:01.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-12 00:04:01.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored wh

KeyboardInterrupt: 

In [28]:
!streamlit run app.py &>/dev/null&

In [29]:
!pip install pyngrok
from pyngrok import ngrok

# Authenticate ngrok
ngrok.set_auth_token("your_ngrok_auth_token")  # Replace with your ngrok token

In [31]:
!pip install pyngrok

In [33]:
from pyngrok import ngrok

# Authenticate ngrok
ngrok.set_auth_token("2suwnNOZzxVLPdDtjAXjhUTAUll_GBUvvMp9UkH3rvJcVhVQ")  # Replace with your actual authtoken

In [36]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2025-02-12 00:11:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 99.83.220.108, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  7.25MB/s    in 1.8s    

2025-02-12 00:11:11 (7.25 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [37]:
!streamlit run app.py &>/dev/null&

In [38]:
public_url = ngrok.connect(addr=8501, proto="http")
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://ce87-34-80-240-156.ngrok-free.app" -> "http://localhost:8501"
